In [1]:
import ROOT

Welcome to JupyROOT 6.12/04


In [6]:
0.5* ROOT.Math.chisquared_quantile(0.68, 1)

0.49447324073901144

In [5]:
0.5*ROOT.Math.chisquared_quantile(0.68, 2)

1.1394342831883648

In [16]:
ROOT.Math.normal_quantile(0.68, 1)

0.4676987991145084

In [20]:
ROOT.Math.normal_cdf(1)

0.8413447460685429

In [21]:
def get_coverage(sigma):
    return 1 - 2*(1 - ROOT.Math.normal_cdf(sigma))

In [22]:
get_coverage(1)

0.6826894921370859

In [23]:
get_coverage(2)

0.9544997361036416

In [29]:
ROOT.Math.chisquared_quantile(get_coverage(4), 1)

15.999999999999998

In [ ]:
def fit(obs, exp, up, down, out_name=None):
    sig_exp = obs - exp
    max_sig = sig_exp * 1.5
    min_sig = sig_exp * 0.5
    sig_var = ROOT.RooRealVar("sig_var", "signal yields", sig_exp, min_sig, max_sig)
    exp_var = ROOT.RooRealVar("exp_var", "expected yields", exp)

    nuis_exp = ROOT.RooRealVar("nuis_exp", "nuisance parameter", 0., -5., 5.)
    low_values = ROOT.std.vector('double')()
    low_values.push_back(1 - down/exp)
    highValues = ROOT.std.vector('double')()
    highValues.push_back(1 + up/exp)
    exp_sys = ROOT.RooStats.HistFactory.FlexibleInterpVar(
        "exp_sys", "exp_sys",
        ROOT.RooArgList(nuis_exp), 1., low_values, highValues)
    exp_var_sys = ROOT.RooFormulaVar("exp_var_sys", "@0 + @1*@2", ROOT.RooArgList(sig_var, exp_var, exp_sys))
    exp_var_sys.Print()

    obs_var = ROOT.RooRealVar("obs_var", "observed events", obs)
    nll = ROOT.RooFormulaVar("nll", "-1 * @0 * TMath::Log(@1) + @1 + 0.5*@2*@2",
                             ROOT.RooArgList(obs_var, exp_var_sys, nuis_exp))

    # create NLL and minize
    minim = ROOT.RooMinimizer(nll)
    minim.setStrategy(1)
    minim.minimize("Minuit2", ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo())

    minim_nll = nll.getVal()
    sig_list = []
    nll_list = []
    ninputs = 30
    for sig_id in range(ninputs+1):
        sig = min_sig + (max_sig - min_sig)*sig_id/ninputs
        sig_list.append(sig)
        sig_var.setVal(sig)
        sig_var.setConstant()
        minim.minimize("Minuit2", ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo())
        nll_list.append(2*(nll.getVal() - minim_nll))

    sig_var.setVal(0.)
    sig_var.setConstant()
    minim.minimize("Minuit2", ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo())
    significance = math.sqrt(2*(nll.getVal() - minim_nll))
    p0 = ROOT.RooStats.SignificanceToPValue(significance)

    gr = ROOT.TGraph(len(sig_list), array.array('f', sig_list), array.array('f', nll_list))
    c1 = ROOT.TCanvas("c1", "c1", 600, 600)
    gr.Draw("ALP")
    gr.GetYaxis().SetRangeUser(0., gr.GetYaxis().GetXmax())
    if out_name is not None:
        c1.SaveAs(out_name)
    else:
        c1.SaveAs("{:.2f}_{:.2f}_{:.2f}.pdf".format(obs, exp, up))

    print "sigma: ", significance
    print "p0: ", p0
return math.sqrt(2*(nll.getVal() - minim_nll)), p0

In [37]:
import ROOT
ROOT.gROOT.SetBatch()

import math
import array

def compatability(obs, exp, up, down, out_name=None):
    sig_exp = obs - exp
    max_sig = sig_exp * 1.5
    #min_sig = sig_exp * 0.5
    min_sig = 0
    sig_var = ROOT.RooRealVar("sig_var", "signal yields", sig_exp, min_sig, max_sig)
    exp_var = ROOT.RooRealVar("exp_var", "expected yields", exp)

    nuis_exp = ROOT.RooRealVar("nuis_exp", "nuisance parameter", 0., -5., 5.)
    low_values = ROOT.std.vector('double')()
    low_values.push_back(1 - down/exp)
    highValues = ROOT.std.vector('double')()
    highValues.push_back(1 + up/exp)
    exp_sys = ROOT.RooStats.HistFactory.FlexibleInterpVar(
        "exp_sys", "exp_sys",
        ROOT.RooArgList(nuis_exp), 1., low_values, highValues)
    exp_var_sys = ROOT.RooFormulaVar("exp_var_sys", "@0 + @1*@2", ROOT.RooArgList(sig_var, exp_var, exp_sys))
    exp_var_sys.Print()

    obs_var = ROOT.RooRealVar("obs_var", "observed events", obs)
    nll = ROOT.RooFormulaVar("nll", "-1 * @0 * TMath::Log(@1) + @1 + 0.5*@2*@2",
                             ROOT.RooArgList(obs_var, exp_var_sys, nuis_exp))

    # create NLL and minize
    minim = ROOT.RooMinimizer(nll)
    minim.setStrategy(2)
    minim.minimize("Minuit2", ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo())

    minim_nll = nll.getVal()
    sig_list = []
    nll_list = []
    ninputs = 30
    for sig_id in range(ninputs+1):
        sig = min_sig + (max_sig - min_sig)*sig_id/ninputs
        sig_list.append(sig)
        sig_var.setVal(sig)
        sig_var.setConstant()
        minim.minimize("Minuit2", ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo())
        nll_list.append(2*(nll.getVal() - minim_nll))

    sig_var.setVal(0.)
    sig_var.setConstant()
    minim.minimize("Minuit2", ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo())
    significance = math.sqrt(2*(nll.getVal() - minim_nll))
    p0 = ROOT.RooStats.SignificanceToPValue(significance)

    gr = ROOT.TGraph(len(sig_list), array.array('f', sig_list), array.array('f', nll_list))
    c1 = ROOT.TCanvas("c1", "c1", 600, 600)
    gr.Draw("ALP")
    gr.GetYaxis().SetRangeUser(0., gr.GetYaxis().GetXmax())
    if out_name is not None:
        c1.SaveAs(out_name)
    else:
        c1.SaveAs("{:.2f}_{:.2f}_{:.2f}.pdf".format(obs, exp, up))

    print "sigma: ", significance
    print "p0: ", p0
    return math.sqrt(2*(nll.getVal() - minim_nll)), p0

In [38]:
compatability(15, 10, 3, 3)

sigma:  1.47103815153
p0:  0.0706403997669


(1.4710381515259618, 0.07064039976686041)

RooFormulaVar::exp_var_sys[ actualVars=(sig_var,exp_var,exp_sys) formula="@0 + @1*@2" ] = 15
[#1] INFO:Minization -- RooFormulaVar::defaultErrorLevel(nll) WARNING: Formula contains neither RooNLLVar nor RooChi2Var server, using default level of 1.0
Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 2
MnSeedGenerator: for initial parameters FCN = -25.62075301653
MnSeedGenerator: Initial state:   - FCN =  -25.62075301653 Edm =   6.3035e-12 NCalls =      9
MnSeedGenerator: run Hesse - new state:   - FCN =  -25.62075301653 Edm =  4.85513e-20 NCalls =     25
VariableMetric: start iterating until Edm is < 0.002
VariableMetric: Initial state   - FCN =  -25.62075301653 Edm =  4.85513e-20 NCalls =     25
VariableMetric: Iteration #   0 - FCN =  -25.62075301653 Edm =  4.85513e-20 NCalls =     25
VariableMetric: Iteration #   1 - FCN =  -25.62075301653 Edm =  4.85513e-20 NCalls =     26
VariableMetric: After Hessian   - FCN =  -25.62075301653 Edm =  4.85517e-20 NCalls

Info in <Minuit2>: VariableMetricBuilder: no improvement in line search
Info in <TCanvas::Print>: pdf file 15.00_10.00_3.00.pdf has been created
